In [2]:
import numpy as np
import pandas as pd
import requests
import json
import rdflib
from tqdm import tqdm
path = './tkgl-smallpedia/tkgl-smallpedia_'
edge_list = pd.read_csv(path+'edgelist.csv')
edge_list_named = pd.read_csv(path+'edgelist_named.csv').drop('Unnamed: 0', axis=1, inplace=False)
edgde_list_named = edge_list_named
edge_list_named_pred = pd.read_csv('./llm_output.csv').drop('Unnamed: 0', axis=1, inplace=False)

In [3]:
def extract_train(df, last_year):
  df_train = df[df['ts'] < last_year]
  df_test = df[df['ts'] == last_year + 1]
  return df_train, df_test

df_train, df_test = extract_train(edgde_list_named, 1925)

In [48]:
def create_ranked_relations(pred_df, gt_df):
    """
    Creates a new dataframe with the rank of the correct relation in the predicted relations list.
    
    Parameters:
    pred_df: DataFrame with columns 'head', 'tail', 'pred_rel' (list of predicted relations)
    gt_df: DataFrame with columns 'head', 'tail', 'relation' (ground truth relation)
    
    Returns:
    DataFrame with columns 'head', 'tail', 'rank' (-1 if not in predictions)
    """
    results = []
    
    # Create a dictionary for faster lookup of ground truth relations
    gt_dict = {(row['head'], row['tail']): row['relation_type'] 
               for _, row in gt_df.iterrows()}
    
    
    # Process each prediction
    for _, row in pred_df.iterrows():
        head = row['head']
        tail = row['tail']
        pred_relations = np.array(eval(row['pred_rel']))
        
        # Get ground truth relation if it exists
        true_relation = gt_dict.get((head, tail))
        
        
        if true_relation is None:
            # If no ground truth exists for this pair
            rank = -1
        else:
            # Find position of true relation in predictions
            if true_relation in pred_relations:
                rank = np.argwhere(pred_relations==true_relation) + 1
                rank = rank[0][0]
            else:
                rank = -1  
        
        results.append({
            'head': head,
            'tail': tail,
            'rank': rank
        })
    
    return pd.DataFrame(results)

# Usage example:
ranked_df = create_ranked_relations(edge_list_named_pred, df_test)

In [57]:
def compute_MRR(df_ranked):
    MRR = 0
    for rank in df_ranked['rank']:
        if rank == -1:
            continue
        MRR +=1/rank
    return MRR/len(ranked_df)